In [10]:
!pip install folium
import folium

In [21]:
#Read dataframes
import pandas as pd
airlines_df = pd.read_csv('/Users/hannapedersen/Documents/NOVA/T3/Advanced Programming/Assignment/flight_data/airlines.csv')
airplanes_df = pd.read_csv('/Users/hannapedersen/Documents/NOVA/T3/Advanced Programming/Assignment/flight_data/airplanes.csv', on_bad_lines='skip')
airports_df = pd.read_csv('/Users/hannapedersen/Documents/NOVA/T3/Advanced Programming/Assignment/flight_data/airports.csv', on_bad_lines='skip')
routes_df =  pd.read_csv('/Users/hannapedersen/Documents/NOVA/T3/Advanced Programming/Assignment/flight_data/routes.csv', on_bad_lines='skip')


In [22]:

def departing_flights_airport(self, airport, internal=False):
    """
    Retrieve and plot departing flights from a given airport on a map, displaying each route uniquely.

    Args:
        airport (str): The IATA code of the airport from which departing flights are to be retrieved.
        internal (bool, optional): If set to True, only internal flights (flights within the same country) are displayed. Defaults to False, which shows all departing flights.

    Returns:
        A Folium map object with plotted departing flights or None if no flights are found.

    Description:
        This function joins route data with airport information to retrieve detailed flight departures from the specified airport. It includes both the source and destination information, such as country and geographical coordinates (latitude and longitude).
        
        The function then filters the flights based on the specified airport code and whether the flight is internal. For internal flights, it further filters to include only those flights where the source and destination countries match.
        
        Unique departing flights are identified by ensuring no duplicate destination airports are plotted. This helps in visualizing each destination uniquely from the specified source airport.
        
        The method visualizes the flights on a map, marking the source airport in green and the destinations in red. Lines connecting the source and destination represent the flight routes. The map is centered around the source airport and adjusted for an appropriate zoom level to capture all routes effectively.
        
        If no flights are found, a message is printed, and the function returns None. Otherwise, it returns a Folium map object that can be displayed in Jupyter notebooks or saved as an HTML file for web viewing.
    """

    # Join routes with airports to find Source and Destination information
    airport_info_1 = self.routes_df.join(self.airports_df.set_index('IATA')[['Country', 'Latitude', 'Longitude']], on='Source airport', rsuffix='_source')
    airport_info_1.rename(columns={'Country': 'Source Country', 'Latitude': 'Source Latitude', 'Longitude': 'Source Longitude'}, inplace=True)
    airport_info_2 = airport_info_1.join(self.airports_df.set_index('IATA')[['Country', 'Latitude', 'Longitude']], on='Destination airport', rsuffix='_dest')
    airport_info_2.rename(columns={'Country_dest': 'Destination Country', 'Latitude': 'Destination Latitude', 'Longitude': 'Destination Longitude'}, inplace=True)
    
    # Filter flights based on the given source airport
    source_flights = airport_info_2[airport_info_2['Source airport'] == airport]
    source_flights_unique = source_flights.drop_duplicates(subset=['Destination airport'])


    if internal:
        # Filter for internal flights (destination in the same country)
        source_flights_unique = source_flights_unique[source_flights_unique['Source Country'] == source_flights_unique['Destination Country']]
        print(f"Internal flights from {airport} to destinations in the same country:")
    else:
        print(f"All flights from {airport}:")

    # Check if there are any flights to display
    if not source_flights_unique.empty:
        # Create a base map centered around the source airport
        m = folium.Map(location=[source_flights_unique.iloc[0]['Source Latitude'], source_flights_unique.iloc[0]['Source Longitude']], zoom_start=5)
            
        # Plot each route
        for idx, row in source_flights_unique.iterrows():
            source = [row['Source Latitude'], row['Source Longitude']]
            destination = [row['Destination Latitude'], row['Destination Longitude']]
            folium.Marker(source, popup=row['Source airport'], icon=folium.Icon(color="green")).add_to(m)
            folium.Marker(destination, popup=row['Destination airport'], icon=folium.Icon(color="red")).add_to(m)
            folium.PolyLine([source, destination], color="blue", weight=2.5, opacity=1).add_to(m)
            
        # Show the map
        return m
    else:
        print(f"No flights found from {airport}.")
        return None


In [23]:
departing_flights_airport('ATL', 'ATL')

AttributeError: 'str' object has no attribute 'routes_df'